In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [20]:
URL="https://nguoinoitieng.tv"
headers = {
  'authority': 'nguoinoitieng.tv',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'accept-language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7',
  'cache-control': 'max-age=0',
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Linux"',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [21]:
def get_profile(url):
    try:
        res = requests.get(f"{URL}{url}", headers=headers)
        doc = BeautifulSoup(res.text, "html.parser")
        # find div with class="motangan"
        short_bio_div = doc.find("div", class_="motangan")
        name = short_bio_div.find("h2").text
        # get age if not exist get birthday
        # find p with form like <p><b>Tuổi:</b> 30</p> (regex form <p><b>Tuổi:</b> {age}</p>)
        age_tag = doc.find("b", string="Tuổi:")
        age = age_tag.next_sibling.text if age_tag else None
        # find p with form like <p><b>Nơi sinh: </b> Quảng Ngãi</p>
        place_of_birth_tag = doc.find("b", string="Nơi sinh: ")
        place_of_birth = place_of_birth_tag.next_sibling.text if place_of_birth_tag else None
        content_left_div = doc.find("div", id="content-left")
        # find first h1 in content_left_div
        first_h1 = content_left_div.find("h1")
        keyword = first_h1.text
        
        yield {
            "name": name,
            "age": age,
            "keyword": keyword,
            "place_of_birth": place_of_birth
        }
    except Exception as e:
        print(e)
        yield {
            "name": None,
            "age": None,
            "keyword": None,
            "place_of_birth": None
        }

In [22]:
# load data (giai_tri_link.csv)
df = pd.read_csv('giai_tri_link.csv')
# get url column
urls = df['url']

In [23]:
# get profiles
profiles_list = []

In [24]:
def get_profiles():
    for url in urls:
        print(url)
        yield from get_profile(url) # yield from is a new syntax in Python 3.3 that allows you to yield all values from an iterable without having to loop over it.

In [25]:
profiles = get_profiles()

In [26]:
for profile in profiles:
    profiles_list.append(profile)

/nghe-nghiep/ca-si/son-tung/k8


/nghe-nghiep/ca-si/hari-won/lc
/nghe-nghiep/ca-si/khoi-my/du
/nghe-nghiep/ca-si/ho-quang-hieu/c0
/nghe-nghiep/ca-si/thuy-tien/in
/nghe-nghiep/ca-si/dong-nhi/br
/nghe-nghiep/ca-si/minh-hang/ac
/nghe-nghiep/ca-si/nhat-kim-anh/d2
/nghe-nghiep/ca-si/ly-hai/e9
/nghe-nghiep/ca-si/bao-thy/q
/nghe-nghiep/ca-si/kelvin-khanh/km
/nghe-nghiep/ca-si/ho-ngoc-ha/cw
/nghe-nghiep/ca-si/ngo-kien-huy/ga
/nghe-nghiep/ca-si/my-tam/b
/nghe-nghiep/ca-si/tuan-hung/jv
/nghe-nghiep/ca-si/nguyen-huong-ly/bemb
/nghe-nghiep/ca-si/le-bao-binh/eo
/nghe-nghiep/ca-si/khong-tu-quynh/dv
/nghe-nghiep/ca-si/dam-vinh-hung/a2
/nghe-nghiep/ca-si/hoang-thuy-linh/lr
/nghe-nghiep/ca-si/karik/p2
/nghe-nghiep/ca-si/dan-truong/a4
/nghe-nghiep/ca-si/luong-bich-huu/ey
/nghe-nghiep/ca-si/noo-phuoc-thinh/gx
/nghe-nghiep/ca-si/phi-nhung/hb
/nghe-nghiep/ca-si/van-mai-huong/k3
/nghe-nghiep/ca-si/suboi/m4
/nghe-nghiep/ca-si/cam-ly/6
/nghe-nghiep/ca-si/phuong-my-chi/hg
/nghe-nghiep/ca-si/bich-phuong/v
/nghe-nghiep/ca-si/ung-hoang-phuc/kb
/

In [27]:
# load list of dict to dataframe
df = pd.DataFrame(profiles_list)
df.head()

,name,age,keyword,place_of_birth
0,Sơn Tùng,29,Ca sĩ Sơn Tùng,Thái Bình
1,Hari Won,38,Ca sĩ Hari Won,Seoul
2,Khởi My,33,Ca sĩ Khởi My,Đồng Nai
3,Hồ Quang Hiếu,37,Ca sĩ Hồ Quang Hiếu,Đắk Lắk
4,Thuỷ Tiên,38,Ca sĩ Thuỷ Tiên,Kiên Giang


In [28]:
df.describe()

,name,age,keyword,place_of_birth
count,4279,4279,4279,4279
unique,4135,127,4235,229
top,Tuyết Mai,28,Hot girl Nguyễn Quỳnh Anh,Hồ Chí Minh
freq,4,302,3,962


In [29]:
# remove duplicate
df = df.drop_duplicates()

In [30]:
df.shape

(4278, 4)

In [31]:
# find two first words in keyword column
df['job'] = df['keyword'].str.split().str[:2].str.join(' ')

In [32]:
df['age'].unique()
# remove first space in age column
df['age'] = df['age'].str.strip()
df['age'].unique()
# get two first number character in age column (regex form {number}{number} or {number} or -{number})
df['age'] = df['age'].str.extract(r'(\d{2}|\d|-{1}\d{2})')
# convert age to float
df['age'] = df['age'].astype(float)
# remove age < 0 and age > 100
df = df[(df['age'] >= 0) & (df['age'] <= 100)]
# remove age = nan (age = None)
df = df[df['age'].notna()]
df['age'].unique()


array([29., 38., 33., 37., 35., 36., 55., 39., 42., 45., 30., 32., 52.,
       34., 47., 49., 53., 20., 28., 26., 43., 46., 31., 23., 50., 41.,
       27., 48., 40., 21., 17., 58., 56., 18., 65., 24., 25., 44., 81.,
       82., 12., 19., 54., 64., 22., 14., 16., 51., 79., 75., 61., 62.,
       15., 67., 89., 76., 70., 57., 68., 87., 74., 78., 72., 13., 77.,
       60., 69., 71., 63., 73., 86., 59., 11., 85., 10., 83., 92., 66.,
       84.])

In [33]:
# convert age to int
df['age'] = df['age'].astype(int)
df['age'].unique()

array([29, 38, 33, 37, 35, 36, 55, 39, 42, 45, 30, 32, 52, 34, 47, 49, 53,
       20, 28, 26, 43, 46, 31, 23, 50, 41, 27, 48, 40, 21, 17, 58, 56, 18,
       65, 24, 25, 44, 81, 82, 12, 19, 54, 64, 22, 14, 16, 51, 79, 75, 61,
       62, 15, 67, 89, 76, 70, 57, 68, 87, 74, 78, 72, 13, 77, 60, 69, 71,
       63, 73, 86, 59, 11, 85, 10, 83, 92, 66, 84])

In [34]:
# drop duplicate again
df = df.drop_duplicates()
df.shape

(4275, 5)

In [35]:
df['job'].unique()

array(['Ca sĩ', 'Hot girl', 'Diễn viên'], dtype=object)

In [36]:
# remove keyword duplicate
df = df.drop_duplicates(subset=['keyword'])
df.shape

(4233, 5)

In [37]:
# show all row where keyword duplicate
df[df.duplicated(subset=['keyword'], keep=False)]

,name,age,keyword,place_of_birth,job


In [39]:
df.head()

,name,age,keyword,place_of_birth,job
0,Sơn Tùng,29,Ca sĩ Sơn Tùng,Thái Bình,Ca sĩ
1,Hari Won,38,Ca sĩ Hari Won,Seoul,Ca sĩ
2,Khởi My,33,Ca sĩ Khởi My,Đồng Nai,Ca sĩ
3,Hồ Quang Hiếu,37,Ca sĩ Hồ Quang Hiếu,Đắk Lắk,Ca sĩ
4,Thuỷ Tiên,38,Ca sĩ Thuỷ Tiên,Kiên Giang,Ca sĩ


In [40]:
df['place_of_birth'].unique()

array(['Thái Bình', 'Seoul', 'Đồng Nai', 'Đắk Lắk', 'Kiên Giang',
       'Hà Nội', 'Hồ Chí Minh', 'Trà Vinh', 'Tiền Giang',
       'Thừa Thiên Huế', 'Đà Nẵng', 'Quảng Nam', 'Hà Nam', 'Quảng Đông',
       'Gia Lai', 'Bình Định', 'Quảng Ninh', 'An Giang', 'Bắc Ninh',
       'Bắc Giang', 'Khánh Hòa', 'Ninh Bình', 'Hà Tĩnh', 'Thanh Hóa',
       'Hải Dương', 'New York City', 'Nghệ An', 'Nước Jamaica',
       'Vĩnh Long', 'Cà Mau', 'Madrid', 'Tứ Xuyên', 'Hồ Nam',
       'Nước Trung quốc', 'Thượng Hải', 'Sóc Trăng', 'Quảng Trị',
       'Hải Phòng', 'Quebec', 'Lâm Đồng', 'Liêu Ninh', 'Nam Định',
       'Hong Kong', 'Hồ Bắc', 'Bay City', 'Bến Tre', 'Yorkshire',
       'Thái Nguyên', 'Newark', 'An Huy', 'Nước Việt Nam', 'San Juan',
       'Tupelo', 'Mechanicsville', 'South Korea', 'Cam Túc', 'Đài Bắc',
       'Thiên Tân', 'Chiết Giang', 'Phúc Kiến', 'Sơn Đông', 'Adelaide',
       'Liverpool', 'Duluth', 'London', 'Cao Hùng', 'Littleton',
       'Newcastle', 'Quảng Châu', 'California', 'Guerrero

In [44]:
# Load tinh thanh viet nam from danhsachttVN.txt
with open('danhsachttVN.txt', 'r') as f:
    tinh_thanh_vn = f.read().split('\n')
# convert to df
tinh_thanh_vn_df = pd.DataFrame(tinh_thanh_vn, columns=['place_of_birth'])
tinh_thanh_vn_df.shape

(65, 1)

In [47]:
# filter all row where place_of_birth in tinh_thanh_vn_df
df = df[df['place_of_birth'].isin(tinh_thanh_vn_df['place_of_birth'])]
df.shape

(2840, 5)

In [48]:
export_to = "giai_tri.csv"
df.to_csv(export_to, index=False)